In [51]:
import yfinance as yf
import json

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = OpenAI()

In [52]:
def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()

In [53]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retorna_cotacao_acao_historica",
            "description": "Obtém a cotação de um ativo financeiro em um determinado periodo",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Codigo ou ticker da empresa na bolsa brasileira. Ex: ABEV3 para ambev, PETR4 para petrobras.",
                    },
                    "periodo": {
                        "type": "string",
                        "description": "Abreviacao do periodo requerido. Ex: 1d, que corresponde a 1 dia e 1mo, que corresponde a um mês",
                        "enum": ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
                    },
                },
                "required": ["ticker"],
            },
        },
    }
]

funcoes_disponiveis = {
        "retorna_cotacao_acao_historica": retorna_cotacao_acao_historica,
    }

In [69]:
def gerar_resposta(mensagens, model="gpt-3.5-turbo-0125", max_tokens=200, temperature=0.5, tools=tools, tool_choice="auto",):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        tools=tools,
        tool_choice=tool_choice,
    )
    mensagem_resp = resposta.choices[0].message
    tool_calls = mensagem_resp.tool_calls

    if tool_calls:
        mensagens.append(mensagem_resp)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = funcoes_disponiveis[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            mensagens.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
    )
    
    print(f'Assistant: ', end='')
    print(resposta.choices[0].message.content)
    print()
    mensagens.append(
        resposta.choices[0].message.model_dump(exclude_none=True))
    return mensagens

In [70]:
print('Bem vindo ao ChatBot de SYSPWL. Qual seu nome?')
print(f'Digite a qualquer momento "sair", para sair do ChatBot.')
nome = ''
iniciar_chat = False
mensagens =  []

while True:
    nome = input('Digite seu nome aqui: ')
    if nome.lower() == 'sair':
        print('Agradecemos seu contato. Até breve!')
        break
    if nome:
        print(f'Seu nome é {nome}, está correto?')
        confirmacao_nome = input('Digite S para confirmar!')
        if confirmacao_nome.lower() != 's':
            continue
        else:
            iniciar_chat = True
            break


if iniciar_chat:
    print(f'Assistant: Olá {nome}, tudo bem? Em que posso ajudar?')
    while True:
        texto_user = input(f'{nome}: ')
        if texto_user.lower() != 'sair':
            mensagens.append({
                'role': 'user',
                'content': texto_user
            })
            mensagens = gerar_resposta(mensagens)
        else:
            print('Agradecemos seu contato. Até breve!')
            break

Bem vindo ao ChatBot de SYSPWL. Qual seu nome?
Digite a qualquer momento "sair", para sair do ChatBot.
Seu nome é PAULO, está correto?
Assistant: Olá PAULO, tudo bem? Em que posso ajudar?
Assistant: A cotação mais recente do Banco do Brasil (ticker: BBAS3) é de R$ 24,17.

Agradecemos seu contato. Até breve!
